In [ ]:
                                                            #######################
                                                            ######## PRISM ########
                                                            #######################

In [ ]:
"""
Mask PRISM dataset to a box over the Great Lakes Basin and reorder dimensions
Varibale can be prcp, tmin or tmax
--------------------------------------------------------
input : D:\PRISM_t\tmax\nc file 1991-2013\PRISM_tmax_4km_1991_2013.nc
output: D:\PRISM_t\tmax\nc file 1991-2013\PRISM_tmax_4km_GLB_1991_2013.nc
"""

from pathlib import Path
import xarray as xr
import numpy as np

in_nc  = Path(r"D:\PRISM_t\tmin\nc file 1991-2013\PRISM_tmin_4km_1991_2013.nc") # Choose your own input path
out_nc = Path(r"D:\PRISM_t\tmin\nc file 1991-2013\PRISM_tmin_4km_GLB_1991_2013_mod.nc")

# ----------------------------------------------------------------------
# 1) open, rename coordinates
# ----------------------------------------------------------------------
ds = xr.open_dataset(in_nc)

ds = ds.rename({"x": "lon", "y": "lat"})

# ----------------------------------------------------------------------
# 2) build boolean mask for requested window
#    (use .where(..., drop=True) so non-matching rows/cols disappear)
# ----------------------------------------------------------------------
mask = (
    (ds.lon >= -95.5) & (ds.lon <= -72.0) &
    (ds.lat >=  38.5) & (ds.lat <=  50)
)

# Apply mask to every variable that has lat & lon dims
ds = ds.where(mask, drop=True)

# ----------------------------------------------------------------------
# 3) reorder dimensions to (lat, lon, time)
# ----------------------------------------------------------------------
# If lat is descending, leave it that way – CF doesn’t care about order.
wanted_order = ("lat", "lon", "time")
ds = ds.transpose(*wanted_order)

# ----------------------------------------------------------------------
# 4) write NetCDF  (keep compression & nodata from source file)
# ----------------------------------------------------------------------
enc = {
    "tmin": dict(zlib=True, complevel=4, dtype="float32"),
    "time": dict(dtype="int32", zlib=True, complevel=4, _FillValue=None),
}

ds.to_netcdf(
    out_nc,
    mode="w",
    format="NETCDF4",
    engine="netcdf4",
    encoding=enc,
)

print("✅ masked cube saved →", out_nc)
print(ds)

In [ ]:
# ───────────────────────── merge_PRISM_tmin_tmax ─────────────────────
# Merge the Great-Lakes-masked PRISM Tmin + Tmax cubes into ONE NetCDF
#   • input 1 : PRISM_tmin_4km_GLB_1991_2013_mod.nc   (has *only* Tmin)
#   • input 2 : PRISM_tmax_4km_GLB_1991_2013_mod.nc   (has *only* Tmax)
#   • output  : PRISM_tmin_tmax_4km_GLB_1991_2013.nc  (both variables)
# -----------------------------------------------------------------------

from pathlib import Path
import xarray as xr

# ─── paths ──────────────────────────────────────────────────────────────
tmin_nc = Path(r"D:\PRISM_t\tmin\nc file 1991-2013\PRISM_tmin_4km_GLB_1991_2013_mod.nc")  # Choose your own input path
tmax_nc = Path(r"D:\PRISM_t\tmax\nc file 1991-2013\PRISM_tmax_4km_GLB_1991_2013_mod.nc")

out_nc  = Path(r"D:\PRISM_t\tmin-max\nc file 1991-2013\PRISM_tmin_tmax_4km_GLB_1991_2013.nc")
out_nc.parent.mkdir(parents=True, exist_ok=True)

# ─── open & rename variables to canonical names ─────────────────────────
ds_min = xr.open_dataset(tmin_nc)            # expect single var inside
ds_max = xr.open_dataset(tmax_nc)

# whatever they are called, rename to “tmin” / “tmax”
min_var = next(iter(ds_min.data_vars))
max_var = next(iter(ds_max.data_vars))
ds_min = ds_min.rename({min_var: "tmin"})
ds_max = ds_max.rename({max_var: "tmax"})

# ─── make sure grids & calendars coincide  (should, but be safe) ────────
ds_min, ds_max = xr.align(ds_min, ds_max, join="exact")

# ─── merge & copy some global attributes  ───────────────────────────────
ds_merged = xr.merge([ds_min["tmin"], ds_max["tmax"]])
ds_merged.attrs = {**ds_min.attrs, **ds_max.attrs}
ds_merged.attrs["history"] = (
    ds_merged.attrs.get("history", "") +
    " | merged tmin & tmax (PRISM Great-Lakes crop)"
)

# ─── compression / encoding  -------------------------------------------
comp = dict(zlib=True, complevel=4)
enc  = {
    "tmin": comp,
    "tmax": comp,
    "time": dict(dtype="int32", zlib=True, complevel=4, _FillValue=None),
}

# ─── write out  ---------------------------------------------------------
ds_merged.to_netcdf(out_nc, format="NETCDF4", engine="netcdf4", encoding=enc)
print("✅  merged cube written →", out_nc)
print(ds_merged)


In [ ]:
# Resampling MERIT DEM (90 m) to PRISM 4-km grid (0.041666667° × 0.041666667°) - "it can be resampled to any other spatial resolution for other datasets"

import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Paths to input and output DEM files
dem_file   = r"D:\PhD\GLB\DEM\Both\Merged_all_ Clipped.tif"
output_dem = r"D:\PhD\GLB\DEM\Both\GreatLakes_MERIT_PRISM_0.0416667deg.tif"

# Open original DEM
with rasterio.open(dem_file) as src:
    src_crs       = src.crs
    left, bottom, right, top = src.bounds
    src_transform = src.transform

    # --- PRISM spacing: 0.041666667° lon, 0.041666667° lat ---
    xres = yres = 0.041666667   # 1/24 degree  ≈ 4 km

    # Compute transformation for resampling
    transform, width, height = calculate_default_transform(
        src_crs, src_crs,
        src.width, src.height,
        left, bottom, right, top,
        resolution=(xres, yres)
    )

    # Update metadata for the new DEM
    new_meta = src.meta.copy()
    new_meta.update({
        "driver":    "GTiff",
        "height":    height,
        "width":     width,
        "transform": transform,
        "nodata":    -9999
    })

    # Create an empty array for the new DEM
    new_dem_data = np.empty((height, width), dtype=np.float32)

    # Resample and write to new file
    with rasterio.open(output_dem, "w", **new_meta) as dst:
        reproject(
            source        = rasterio.band(src, 1),
            destination   = new_dem_data,
            src_transform = src_transform,
            dst_transform = transform,
            src_crs       = src_crs,
            dst_crs       = src_crs,
            resampling    = Resampling.bilinear
        )
        dst.write(new_dem_data, 1)

print(f"✅ Resampled DEM (0.0416667°×0.0416667°) saved as: {output_dem}")


In [ ]:
# Adding Elevation from the resampled MERIT DEM to the merged nc files of prcp 1991-2013 (Here for PRISM)
import rasterio
import numpy as np
import xarray as xr
from scipy.interpolate import griddata
from tqdm import tqdm

# Paths
resampled_dem       = r"D:\PhD\GLB\DEM\Both\GreatLakes_MERIT_PRISM_0.0416667deg.tif"
PRISM_file            = r"D:\PRISM_ppt_daily\nc file 1991-2013\PRISM_ppt_4km_GLB_1991_2013.nc"
updated_PRISM_file    = r"D:\PhD\GLB\EMDNA(Historical data)\Ensembles\New folder\Ensemble files\PRISM_GLB_Precipitation\PRISM_prcp_GLB_1991-2013_with_Elevation.nc"

print("🔹 Opening PRISM dataset…")
PRISM_ds   = xr.open_dataset(PRISM_file)
PRISM_lats = PRISM_ds["lat"].values
PRISM_lons = PRISM_ds["lon"].values

print("🔹 Reading DEM…")
with rasterio.open(resampled_dem) as src:
    dem_data  = src.read(1)
    transform = src.transform
    height, width = dem_data.shape
    dem_lons = np.linspace(src.bounds.left,  src.bounds.right, width)
    dem_lats = np.linspace(src.bounds.top,   src.bounds.bottom, height)
    dem_lon2d, dem_lat2d = np.meshgrid(dem_lons, dem_lats)
    dem_points = np.column_stack((dem_lon2d.ravel(), dem_lat2d.ravel()))
    dem_values = dem_data.ravel()

# Prepare output elevation grid
grid_elev = np.full((len(PRISM_lats), len(PRISM_lons)), np.nan, dtype=np.float32)

print("🔹 Interpolating elevation onto PRISM grid…")
for i, lat in enumerate(tqdm(PRISM_lats, desc="  Rows")):
    # build query points for this row
    query_pts = np.column_stack((PRISM_lons, np.full_like(PRISM_lons, lat)))
    # nearest‐neighbor for entire row at once
    row_vals = griddata(dem_points, dem_values, query_pts, method="nearest")
    grid_elev[i, :] = row_vals

print("🔹 Assigning elevation to dataset…")
PRISM_ds["elevation"] = (("lat", "lon"), grid_elev)
PRISM_ds.elevation.attrs.update(
    long_name="Surface elevation",
    units="m",
)

print("🔹 Writing updated NetCDF…")
PRISM_ds.to_netcdf(updated_PRISM_file)
print(f"✅ Done! Saved with elevation: {updated_PRISM_file}")

# Up to here, we have a netcdf file for PRISM (prcp or tmin/tmax) that has this variable, lat, lon, and elevation

In [ ]:
                                                            #######################
                                                            ######## EMDNA ########
                                                            #######################

In [ ]:
#Extracting prcp from the masked original EMDNA files

"""
Extract precipitation (prcp) from **masked** EMDNA ensemble files.

For every ensemble ID in ENSEMBLES the script:
  • scans  ROOT_DIR/<ensemble>/Masked/   for *.nc4
  • keeps only the variable 'trange'
  • writes each file to  ROOT_DIR/<ensemble>/Masked/trange/<original>_prcp.nc4
"""

import os
import glob
import xarray as xr

# ─────────────── input settings ───────────────
ROOT_DIR  = r"D:\PhD\GLB\EMDNA(Historical data)\Ensembles\New folder\Ensemble files"
ENSEMBLES = [1, 11, 21, 31, 41, 51, 61, 66, 71, 76, 81, 86, 91, 96, 100] # Any choice of ensemble members can be specified
# ──────────────────────────────────────────────

for ens in ENSEMBLES:
    input_folder  = os.path.join(ROOT_DIR, str(ens), "Masked")
    output_folder = os.path.join(input_folder, "prcp")
    os.makedirs(output_folder, exist_ok=True)

    file_list = glob.glob(os.path.join(input_folder, "*.nc4"))
    print(f"\nEnsemble {ens}: found {len(file_list)} files")

    for file_path in file_list:
        base_name   = os.path.basename(file_path)
        output_file = os.path.join(
            output_folder,
            base_name.replace(".nc4", "_prcp.nc4")
        )

        print(f"  → {base_name}")

        # open, subset, save
        with xr.open_dataset(file_path) as ds:
            ds_prcp = ds[['prcp']]
            ds_prcp.to_netcdf(
                output_file,
                encoding={var: {"zlib": True, "complevel": 4}
                          for var in ds_prcp.data_vars}
            )

print("\n✅ All ensembles processed successfully.")


In [ ]:
#Extracting trange from the masked original EMDNA files

"""
Extracting (trange) from **masked** EMDNA ensemble files.

For every ensemble ID in ENSEMBLES the script:
  • scans  ROOT_DIR/<ensemble>/Masked/   for *.nc4
  • keeps only the variable 'trange'
  • writes each file to  ROOT_DIR/<ensemble>/Masked/trange/<original>_trange.nc4
"""

import os
import glob
import xarray as xr

# ─────────────── input settings ───────────────
ROOT_DIR  = r"D:\PhD\GLB\EMDNA(Historical data)\Ensembles\New folder\Ensemble files"
ENSEMBLES = [1, 11, 21, 31, 41, 51, 61, 66, 71, 76, 81, 86, 91, 96, 100] # Any choice of ensemble members can be specified
# ──────────────────────────────────────────────

for ens in ENSEMBLES:
    input_folder  = os.path.join(ROOT_DIR, str(ens), "Masked")
    output_folder = os.path.join(input_folder, "trange")
    os.makedirs(output_folder, exist_ok=True)

    file_list = glob.glob(os.path.join(input_folder, "*.nc4"))
    print(f"\nEnsemble {ens}: found {len(file_list)} files")

    for file_path in file_list:
        base_name   = os.path.basename(file_path)
        output_file = os.path.join(
            output_folder,
            base_name.replace(".nc4", "_trange.nc4")
        )

        print(f"  → {base_name}")

        # open, subset, save
        with xr.open_dataset(file_path) as ds:
            ds_trange = ds[['trange']]
            ds_trange.to_netcdf(
                output_file,
                encoding={var: {"zlib": True, "complevel": 4}
                          for var in ds_trange.data_vars}
            )

print("\n✅ All ensembles processed successfully.")


In [ ]:
#Extracting tmean from the masked original EMDNA files

"""
Extract daily-mean temperature (tmean) from **masked** EMDNA ensemble files.

For every ensemble ID in ENSEMBLES the script:
  • scans  ROOT_DIR/<ensemble>/Masked/   for *.nc4
  • keeps only the variable 'tmean'
  • writes each file to  ROOT_DIR/<ensemble>/Masked/tmean/<original>_tmean.nc4
"""

import os
import glob
import xarray as xr

# ─────────────── USER SETTINGS ───────────────
ROOT_DIR  = r"D:\PhD\GLB\EMDNA(Historical data)\Ensembles\New folder\Ensemble files"
ENSEMBLES = [1, 11, 21, 31, 41, 51, 61, 66, 71, 76, 81, 86, 91, 96, 100]
# ──────────────────────────────────────────────

for ens in ENSEMBLES:
    input_folder  = os.path.join(ROOT_DIR, str(ens), "Masked")
    output_folder = os.path.join(input_folder, "tmean")
    os.makedirs(output_folder, exist_ok=True)

    file_list = glob.glob(os.path.join(input_folder, "*.nc4"))
    print(f"\nEnsemble {ens}: found {len(file_list)} files")

    for file_path in file_list:
        base_name   = os.path.basename(file_path)
        output_file = os.path.join(
            output_folder,
            base_name.replace(".nc4", "_tmean.nc4")
        )

        print(f"  → {base_name}")

        # open, subset, save
        with xr.open_dataset(file_path) as ds:
            ds_tmean = ds[['tmean']]
            ds_tmean.to_netcdf(
                output_file,
                encoding={var: {"zlib": True, "complevel": 4}
                          for var in ds_tmean.data_vars}
            )

print("\n✅ All ensembles processed successfully.")


In [ ]:
# ───────────── build_emdna_tmin_tmax_only ─────────────
# Making min & max temperature from trange and tmean
"""
Create daily Tmin / Tmax files from EMDNA Tmean + Trange pairs.
Each output NetCDF holds *only* the variables `tmin` and `tmax`.

Folder layout expected
ROOT/<ens>/Masked/
    ├─ tmean\   … *_tmean.nc4
    └─ trange\  … *_trange.nc4
Output goes to
ROOT/<ens>/Masked/tmin_tmax\  … *_tmin_tmax.nc4
"""

import os
import glob
import xarray as xr

ROOT_DIR  = r"D:\PhD\GLB\EMDNA(Historical data)\Ensembles\New folder\Ensemble files"
ENSEMBLES = [1, 11, 21, 31, 41, 51, 61, 66, 71, 76, 81, 86, 91, 96, 100]

for ens in map(str, ENSEMBLES):
    tmean_dir  = os.path.join(ROOT_DIR, ens, "Masked", "tmean")
    trange_dir = os.path.join(ROOT_DIR, ens, "Masked", "trange")
    out_dir    = os.path.join(ROOT_DIR, ens, "Masked", "tmin_tmax")
    os.makedirs(out_dir, exist_ok=True)

    for tmean_fp in glob.glob(os.path.join(tmean_dir, "*_tmean.nc4")):
        stem       = os.path.basename(tmean_fp).replace("_tmean.nc4", "")
        trange_fp  = os.path.join(trange_dir, f"{stem}_trange.nc4")
        if not os.path.exists(trange_fp):
            print(f"[{ens}] missing trange → {stem} skipped")
            continue

        out_fp = os.path.join(out_dir, f"{stem}_tmin_tmax.nc4")

        with xr.open_dataset(tmean_fp) as ds_mean, xr.open_dataset(trange_fp) as ds_rng:
            tmin = ds_mean["tmean"] - ds_rng["trange"] / 2.0
            tmax = ds_mean["tmean"] + ds_rng["trange"] / 2.0

            xr.Dataset(
                {"tmin": tmin, "tmax": tmax},
                attrs=ds_mean.attrs          # keep global metadata
            ).to_netcdf(
                out_fp,
                encoding={v: {"zlib": True, "complevel": 4}
                          for v in ("tmin", "tmax")}
            )
    print(f"[{ens}] ✓ Tmin/Tmax complete")

print("\nAll ensembles processed.")


In [ ]:
# Merging the extracted (tmin-tmax or prcp) to a single nc file from 1991-2013 for each ensemble

"""
Merge yearly Tmin/Tmax files for every ensemble, add CF metadata,
and overwrite the merged NetCDF (1991-2013).

Directory layout assumed
ROOT/<ens>/Masked/tmin_tmax/
    … EMDNA_<ens>_1991_tmin_tmax.nc4
    … EMDNA_<ens>_1992_tmin_tmax.nc4
    …
The script produces (or overwrites) in the same folder:
    EMDNA_<ens>_merged_tmin_tmax_1991_2013.nc
"""

import os, re, glob
import xarray as xr
import pandas as pd

# ─────────── USER SETTINGS ───────────
ROOT_DIR  = r"D:\PhD\GLB\EMDNA(Historical data)\Ensembles\New folder\Ensemble files"
ENSEMBLES = [1, 11, 21, 31, 41, 51, 61, 66, 71, 76, 81, 86, 91, 96, 100]
# ─────────────────────────────────────

YEAR_RE = re.compile(r"(19|20)\d{2}")

def add_cf_attrs(ds: xr.Dataset) -> xr.Dataset:
    ds["lon"]  = ds["lon"].assign_attrs(long_name="longitude",
                                        units="degrees_east")
    ds["lat"]  = ds["lat"].assign_attrs(long_name="latitude",
                                        units="degrees_north")
    ds["tmin"] = ds["tmin"].assign_attrs(long_name="daily minimum 2 m temperature",
                                         units="degC")
    ds["tmax"] = ds["tmax"].assign_attrs(long_name="daily maximum 2 m temperature",
                                         units="degC")
    return ds

for ens in ENSEMBLES:
    ens_dir   = os.path.join(ROOT_DIR, str(ens), "Masked", "tmin_tmax")
    yr_files  = sorted(glob.glob(os.path.join(ens_dir, "*_tmin_tmax.nc4")))
    if not yr_files:
        print(f"[{ens}] no yearly files found → skipped")
        continue

    print(f"[{ens}] merging {len(yr_files)} yearly files")
    datasets = []
    for fp in yr_files:
        year_match = YEAR_RE.search(os.path.basename(fp))
        if not year_match:
            print(f"    ! skipped {os.path.basename(fp)} (no year in name)")
            continue
        year  = int(year_match.group(0))
        dates = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="D")

        ds = xr.open_dataset(fp)
        if ds.dims["time"] != len(dates):
            raise ValueError(f"{os.path.basename(fp)}: expected {len(dates)} days,"
                             f" found {ds.dims['time']}")
        datasets.append(ds.assign_coords(time=dates))

    merged = xr.concat(datasets, dim="time").sortby("time")
    merged = merged.transpose("time", "lat", "lon")
    merged = add_cf_attrs(merged)

    out_fp = os.path.join(ens_dir,
              f"EMDNA_{ens}_merged_tmin_tmax_1991_2013.nc")
    tmp_fp = out_fp + ".tmp"        # safe write, then atomic replace
    merged.to_netcdf(tmp_fp)
    os.replace(tmp_fp, out_fp)
    print(f"    ✓ wrote {out_fp}")

print("\nAll ensembles processed and CF-metadata added.")

# The next step is making the resampled MERIT DEM for EMDNA spatial resolution (0.1) and adding elevation to the prcp and tmin/tmax netcdf files, the same as what has been done for PRISM dataset.

In [ ]:
                                                            ############################
                                                            ######## RDRS v 2.1 ########
                                                            ############################

In [ ]:
# masking RDRS dataset to GLB and converting its coordinates from rotated to regular 
# Here we have precipitation, but the same thing will be done on both tmin and tmax variables

import xarray as xr
import numpy as np
import os

from dask.diagnostics import ProgressBar
from clisops.core import subset
import xesmf as xe
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# For nicer lat/lon tick formatting
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

###################################################
#DEFINE INPUT AND OUTPUT
input_file  = r"D:\RDRS-V2.1\pr_day_GovCan_rdrs-v21_20120101-20151231.nc"
out_subset  = r"D:\RDRS-V2.1\subset\pr_subset_rotated_20120101-20151231.nc"
out_regrid  = r"D:\RDRS-V2.1\subset\pr_subset_regular_20120101-20151231.nc"

# Ensure output directories exist
os.makedirs(os.path.dirname(out_subset), exist_ok=True)

###################################################
#OPEN THE DATASET WITH CHUNKS
print("Opening dataset with rotated coords...")
ds = xr.open_dataset(
    input_file,
    chunks=dict(time=1461, rlon=50, rlat=50)
)
print(ds)

###################################################
#SUBSET TO BOUNDING BOX (in rotated coords, using lat/lon)
min_lon, max_lon = -95.5, -72
min_lat, max_lat = 38.5, 52.5

print("\nLoading lat/lon arrays to memory...")
ds["lat"] = ds["lat"].load()
ds["lon"] = ds["lon"].load()

print(f"Subsetting to bounding box lon=[{min_lon},{max_lon}], lat=[{min_lat},{max_lat}]...")
ds_subset = subset.subset_bbox(
    ds,
    lon_bnds=[min_lon, max_lon],
    lat_bnds=[min_lat, max_lat]
)

print("\nPerforming the subset with dask progress bar...")
with ProgressBar():
    ds_subset = ds_subset.load()

# Save the subset
ds_subset.to_netcdf(out_subset)
print(f"Subset (still rotated) saved to: {out_subset}")

print(ds_subset.rotated_pole)
print(ds_subset.rotated_pole.attrs)

###################################################
#REGRID THE SUBSET TO A REGULAR LAT-LON GRID
print("\nDefining a regular lat-lon grid for regridding...")

target_lats = np.arange(min_lat, max_lat + 0.01, 0.1)
target_lons = np.arange(min_lon, max_lon + 0.01, 0.1)

ds_out = xr.Dataset(
    {
        "lat": (["lat"], target_lats),
        "lon": (["lon"], target_lons),
    }
)

print("Building the xESMF regridder...")
regridder = xe.Regridder(
    ds_subset,
    ds_out,
    method="bilinear",
    reuse_weights=False
)

print("Applying the regridder to 'pr' variable with progress bar...")
with ProgressBar():
    pr_reg = regridder(ds_subset["pr"].load())

ds_reg = pr_reg.to_dataset(name="pr")

# Copy original variable attributes so we can use them in plotting
for attr_name in ["long_name", "units"]:
    if attr_name in ds_subset["pr"].attrs:
        ds_reg["pr"].attrs[attr_name] = ds_subset["pr"].attrs[attr_name]

ds_reg.to_netcdf(out_regrid)
print(f"Regridded data saved to: {out_regrid}")

###################################################
#QUICK PLOT: ORIGINAL (ROTATED) VS. REGRIDDING RESULT
print("\nPlotting the original subset (rotated) and the final regridded subset...")

time_idx = 0
fig = plt.figure(figsize=(10, 8))

# Safely extract rotated pole attributes, fallback if missing
pole_lon = ds_subset.rotated_pole.attrs.get("grid_north_pole_longitude", 87.5970)
pole_lat = ds_subset.rotated_pole.attrs.get("grid_north_pole_latitude", 31.7583)
rotpole = ccrs.RotatedPole(
    pole_longitude=float(pole_lon),
    pole_latitude=float(pole_lat)
)

# PROJECTION for the map: PlateCarree
proj_pc = ccrs.PlateCarree()

#######################
# Original subset
#######################
ax1 = fig.add_subplot(2, 1, 1, projection=proj_pc)
ax1.set_title("Original Subset on Rotated Grid (plotted via RotatedPole transform)")
ax1.set_extent([min_lon, max_lon, min_lat, max_lat], crs=proj_pc)

mesh1 = ax1.pcolormesh(
    ds_subset.rlon,
    ds_subset.rlat,
    ds_subset.pr.isel(time=time_idx),
    transform=rotpole  # Tells Cartopy how to interpret these data coords
)
ax1.coastlines()

# Turn on lat/lon grid labels
gl1 = ax1.gridlines(draw_labels=True, alpha=0.5, linestyle="--")
gl1.xlabels_top = False
gl1.ylabels_right = False
gl1.xformatter = LONGITUDE_FORMATTER
gl1.yformatter = LATITUDE_FORMATTER

cb1 = plt.colorbar(mesh1, orientation="horizontal", pad=0.08, ax=ax1)
pr_long_name = ds_subset["pr"].attrs.get("long_name", "Precipitation")
pr_units     = ds_subset["pr"].attrs.get("units", "kg m-2 s-1")
cb1.set_label(f"{pr_long_name} ({pr_units})")

#######################
# Regridded subset
#######################
ax2 = fig.add_subplot(2, 1, 2, projection=proj_pc)
ax2.set_title("Regridded Subset (Regular Lat-Lon)")
ax2.set_extent([min_lon, max_lon, min_lat, max_lat], crs=proj_pc)

mesh2 = ax2.pcolormesh(
    ds_reg.lon,
    ds_reg.lat,
    ds_reg.pr.isel(time=time_idx),
    transform=proj_pc
)
ax2.coastlines()

# Turn on lat/lon grid labels
gl2 = ax2.gridlines(draw_labels=True, alpha=0.5, linestyle="--")
gl2.xlabels_top = False
gl2.ylabels_right = False
gl2.xformatter = LONGITUDE_FORMATTER
gl2.yformatter = LATITUDE_FORMATTER

cb2 = plt.colorbar(mesh2, orientation="horizontal", pad=0.08, ax=ax2)
pr_long_name = ds_reg["pr"].attrs.get("long_name", "Precipitation")
pr_units     = ds_reg["pr"].attrs.get("units", "kg m-2 s-1")
cb2.set_label(f"{pr_long_name} ({pr_units})")

plt.tight_layout()
plt.show()
print("All done!")


In [ ]:
# Merging the regular grid nc files to one single netcdf files of RDRS
# Here we have precipitation, but the same thing will be done on both tmin and tmax variables

import xarray as xr
import os
import glob

###############################################
# 1) Define paths
###############################################
# Input folder containing your 7 netCDF files
input_folder = r"D:\RDRS-V2.1\subset\Regular coordinates"

# Output file where we’ll save the merged dataset
output_file = r"D:\RDRS-V2.1\subset\Regular coordinates\merged_prcp_1988-2015_mm_day.nc"

###############################################
# 2) Open all netCDF files at once
###############################################
# We assume all relevant files match a pattern like "*.nc"
file_pattern = os.path.join(input_folder, "*.nc")
nc_files = sorted(glob.glob(file_pattern))
print("Found files:")
for f in nc_files:
    print(f)

# Open and combine them by time (assuming each file spans a unique time chunk)
ds = xr.open_mfdataset(nc_files, combine="by_coords")

###############################################
# 3) Rename 'pr' to 'prcp'
###############################################
# Check if 'pr' exists
if "pr" not in ds.variables:
    raise ValueError("Variable 'pr' not found in the datasets!")

ds = ds.rename({"pr": "prcp"})

###############################################
# 4) Convert from kg m-2 s-1 to mm/day
###############################################
# Explanation: 1 kg m-2 s-1 = 1 mm/s of water flux = 86400 mm/day
# because 1 day = 86400 seconds
ds["prcp"] = ds["prcp"] * 86400.0

# Update attributes
ds["prcp"].attrs["units"] = "mm/day"
old_long_name = ds["prcp"].attrs.get("long_name", "")
ds["prcp"].attrs["long_name"] = f"{old_long_name} (converted to mm/day)"

###############################################
# 5) Save to a new netCDF file
###############################################
ds.to_netcdf(output_file)
print(f"\nMerged and converted dataset saved to:\n{output_file}")

# The next step is making the resampled MERIT DEM for RDRS spatial resolution (0.1) and adding elevation to the prcp and tmin/tmax netcdf files, the same as what has been done for PRISM dataset.

In [1]:
# We have the same procedure for ERA5, MERRA-2, and CHIRPS datasets:
#1. Masking to a box over the Great Lakes Basin
#2. Merging the files to a period that includes at least 1991-2012 (daily)
#3. Adding elevation to their netcdf files so that the dimensions of each dataset include time, lat, lon, elevation